# Visualizing Aggregated Analytics

We used spark to get the average velocity, average groupsize, and other nice analytics
from the Spark EMR cluster.

In [1]:
import pandas as pd
import numpy as np
import os

from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, HoverTool

In [36]:
df = pd.read_csv('../full_paired-10mins.csv')

In [37]:
df[:3]

,Unnamed: 0,window,sum(num_people),sum(num_groups),sum(sum_velocities),sum(num_velocities),avg(num_people),x_centers,y_centers
0,0,"Row(start=datetime.datetime(2019, 4, 10, 0, 20...",1,1,0.000000,0,0.001389,[0.449],[0.335]
1,1,"Row(start=datetime.datetime(2019, 4, 10, 0, 30...",7,7,0.014000,1,0.005833,"[0.709, 0.691, 0.718, 0.718, 0.455, 0.421, 0.812]","[0.339, 0.362, 0.459, 0.466, 0.731, 0.764, 0.56]"
2,2,"Row(start=datetime.datetime(2019, 4, 10, 0, 40...",66,36,0.714577,33,0.055000,"[0.694, 0.691, 0.693, 0.7, 0.688, 0.688, 0.685...","[0.334, 0.337, 0.331, 0.33, 0.334, 0.329, 0.33..."


In [38]:
df

,Unnamed: 0,window,sum(num_people),sum(num_groups),sum(sum_velocities),sum(num_velocities),avg(num_people),x_centers,y_centers
0,0,"Row(start=datetime.datetime(2019, 4, 10, 0, 20...",1,1,0.000000,0,0.001389,[0.449],[0.335]
1,1,"Row(start=datetime.datetime(2019, 4, 10, 0, 30...",7,7,0.014000,1,0.005833,"[0.709, 0.691, 0.718, 0.718, 0.455, 0.421, 0.812]","[0.339, 0.362, 0.459, 0.466, 0.731, 0.764, 0.56]"
2,2,"Row(start=datetime.datetime(2019, 4, 10, 0, 40...",66,36,0.714577,33,0.055000,"[0.694, 0.691, 0.693, 0.7, 0.688, 0.688, 0.685...","[0.334, 0.337, 0.331, 0.33, 0.334, 0.329, 0.33..."
3,3,"Row(start=datetime.datetime(2019, 4, 10, 0, 50...",40,35,0.181781,15,0.033333,"[0.458, 0.606, 0.6, 0.48, 0.528, 0.601, 0.615,...","[0.339, 0.264, 0.263, 0.805, 0.763, 0.682, 0.6..."
4,4,"Row(start=datetime.datetime(2019, 4, 10, 1, 0)...",106,56,1.592606,66,0.088333,"[0.853, 0.846, 0.837, 0.83, 0.823, 0.815, 0.80...","[0.53, 0.532, 0.534, 0.539, 0.542, 0.546, 0.55..."
5,5,"Row(start=datetime.datetime(2019, 4, 10, 1, 10...",1,1,0.000000,0,0.000833,[0.451],[0.327]
6,6,"Row(start=datetime.datetime(2019, 4, 10, 1, 20...",13,12,0.029189,4,0.010934,"[0.642, 0.621, 0.616, 0.597, 0.583, 0.429, 0.4...","[0.538, 0.565, 0.575, 0.593, 0.653, 0.789, 0.3..."
7,7,"Row(start=datetime.datetime(2019, 4, 10, 1, 30...",20,16,0.146305,11,0.016667,"[0.456, 0.454, 0.427, 0.434, 0.429, 0.439, 0.4...","[0.331, 0.33, 0.259, 0.255, 0.255, 0.264, 0.26..."
8,8,"Row(start=datetime.datetime(2019, 4, 10, 1, 40...",101,66,1.216250,59,0.084167,"[0.678, 0.709, 0.705, 0.693, 0.687, 0.705, 0.6...","[0.378, 0.486, 0.487, 0.503, 0.5, 0.499, 0.501..."
9,9,"Row(start=datetime.datetime(2019, 4, 10, 1, 50...",1,1,0.000000,0,0.000833,[0.393],[0.346]


Lets first plot the average velocity.
Currently the window is in the format of a string.
We want to make it in the units of datetime, where datetime
is when the windows starts.

In [39]:
df.window[0]

'Row(start=datetime.datetime(2019, 4, 10, 0, 20), end=datetime.datetime(2019, 4, 10, 0, 30))'

Let's take off the row and the parens, and `start=` and `end=`.

In [40]:
start, end = df.window[0][4:-1].replace('start=', '').replace('end=', '').split(', datetime.')
start = start.replace("datetime.", "")
end = end.replace('datetime.', '')
print(start)
print(end)

datetime(2019, 4, 10, 0, 20)
datetime(2019, 4, 10, 0, 30)


In [41]:
from datetime import datetime
def filter_window_row(row):
    start, end = row[4:-1].replace('start=', '').replace('end=', '').split(', datetime.')
    start = start.replace("datetime.", "")
    end = end.replace('datetime.', '')
    return eval(start)

In [42]:
df['start'] = df.window.map(filter_window_row)
df['start_string'] = df['start'].map(lambda timestamp: timestamp.strftime("%b %d %Y %H:%M:%S %p"))
df.index.name = 'index'

In [43]:
df[:3]

,Unnamed: 0,window,sum(num_people),sum(num_groups),sum(sum_velocities),sum(num_velocities),avg(num_people),x_centers,y_centers,start,start_string
index,,,,,,,,,,,
0,0,"Row(start=datetime.datetime(2019, 4, 10, 0, 20...",1,1,0.000000,0,0.001389,[0.449],[0.335],2019-04-10 00:20:00,Apr 10 2019 00:20:00 AM
1,1,"Row(start=datetime.datetime(2019, 4, 10, 0, 30...",7,7,0.014000,1,0.005833,"[0.709, 0.691, 0.718, 0.718, 0.455, 0.421, 0.812]","[0.339, 0.362, 0.459, 0.466, 0.731, 0.764, 0.56]",2019-04-10 00:30:00,Apr 10 2019 00:30:00 AM
2,2,"Row(start=datetime.datetime(2019, 4, 10, 0, 40...",66,36,0.714577,33,0.055000,"[0.694, 0.691, 0.693, 0.7, 0.688, 0.688, 0.685...","[0.334, 0.337, 0.331, 0.33, 0.334, 0.329, 0.33...",2019-04-10 00:40:00,Apr 10 2019 00:40:00 AM


These are now string with the datetime object.

## Compute average statistics (instead of in PySpark) due to the inability of udfs to be parallelizable over multiple columns

In [49]:
df[:3]

,Unnamed: 0,window,sum(num_people),sum(num_groups),sum(sum_velocities),sum(num_velocities),avg(num_people),x_centers,y_centers,start,start_string,avg_group_size,avg_velocity
index,,,,,,,,,,,,,
0,0,"Row(start=datetime.datetime(2019, 4, 10, 0, 20...",1,1,0.000000,0,0.001389,[0.449],[0.335],2019-04-10 00:20:00,Apr 10 2019 00:20:00 AM,1.000000,NaN
1,1,"Row(start=datetime.datetime(2019, 4, 10, 0, 30...",7,7,0.014000,1,0.005833,"[0.709, 0.691, 0.718, 0.718, 0.455, 0.421, 0.812]","[0.339, 0.362, 0.459, 0.466, 0.731, 0.764, 0.56]",2019-04-10 00:30:00,Apr 10 2019 00:30:00 AM,1.000000,0.014000
2,2,"Row(start=datetime.datetime(2019, 4, 10, 0, 40...",66,36,0.714577,33,0.055000,"[0.694, 0.691, 0.693, 0.7, 0.688, 0.688, 0.685...","[0.334, 0.337, 0.331, 0.33, 0.334, 0.329, 0.33...",2019-04-10 00:40:00,Apr 10 2019 00:40:00 AM,1.833333,0.021654


In [64]:
def get_averages(df):
    df = df
    df['avg_group_size'] = 1.*df['sum(num_people)'] / df['sum(num_groups)']
    df['avg_velocity'] = 1.*df['sum(sum_velocities)'] / df['sum(num_velocities)']
    df.rename(columns={'avg(num_people)': 'avg_num_people'}, inplace=True)
    
    # Some windows have 0 groups or velocities, so replace NaNs with 0.
    df.fillna(0.)
    return df

In [65]:
get_averages(df)
df[:3]

,Unnamed: 0,window,sum(num_people),sum(num_groups),sum(sum_velocities),sum(num_velocities),avg_num_people,x_centers,y_centers,start,start_string,avg_group_size,avg_velocity
index,,,,,,,,,,,,,
0,0,"Row(start=datetime.datetime(2019, 4, 10, 0, 20...",1,1,0.000000,0,0.001389,[0.449],[0.335],2019-04-10 00:20:00,Apr 10 2019 00:20:00 AM,1.000000,NaN
1,1,"Row(start=datetime.datetime(2019, 4, 10, 0, 30...",7,7,0.014000,1,0.005833,"[0.709, 0.691, 0.718, 0.718, 0.455, 0.421, 0.812]","[0.339, 0.362, 0.459, 0.466, 0.731, 0.764, 0.56]",2019-04-10 00:30:00,Apr 10 2019 00:30:00 AM,1.000000,0.014000
2,2,"Row(start=datetime.datetime(2019, 4, 10, 0, 40...",66,36,0.714577,33,0.055000,"[0.694, 0.691, 0.693, 0.7, 0.688, 0.688, 0.685...","[0.334, 0.337, 0.331, 0.33, 0.334, 0.329, 0.33...",2019-04-10 00:40:00,Apr 10 2019 00:40:00 AM,1.833333,0.021654


## Visualization

In [57]:
HEIGHT = 360
WIDTH = 640

In [18]:
# Bokeh Library
from bokeh.io import output_file, output_notebook
from bokeh.models.widgets import Tabs, Panel
from bokeh.plotting import show

# Output to file
# output_notebook()

In [68]:
hover = HoverTool()
hover.tooltips = [('Timestamp', '@start_string'),
                  ('Average Number of People', '@avg_num_people'),]

num_people_graph = figure(title="Number of People in Science Center Plaza",
    plot_height=500, 
    plot_width=900,
    # y_range=(0, 20),
    match_aspect=True,
    tools=[hover, "pan,reset,wheel_zoom"])

num_people_graph.vbar(x='index',
       top='avg_num_people',
       width=0.9,
       color='red',
       source=ColumnDataSource(df))

num_people_graph.xaxis.axis_label = "Time Window (Hover for Timestamp)"
num_people_graph.yaxis.axis_label = "Average Number of People in Camera"

# avg_velocity_graph = figure(title="Average Velocity",
#     plot_height=500, 
#     plot_width=900,
#     match_aspect=True,
#     y_range=(0, 1),
#     tools=[hover, "pan,reset,wheel_zoom"])

# avg_velocity_graph.vbar(x='index',
#        top='avg_velocity',
#        width=0.9,
#        color='blue',
#        source=ColumnDataSource(df))

# avg_group_size_graph = figure(title="Stuff",
#     plot_height=500, 
#     plot_width=900,
#     match_aspect=True,
#     y_range=(0, 4),
#     tools=[hover, "pan,reset,wheel_zoom"])

# avg_group_size_graph.vbar(x='index',
#        top='avg_group_size',
#        width=0.9,
#        color='green',
#        source=ColumnDataSource(df))


panel1 = Panel(child=num_people_graph, title='Average Number of People')
# panel2 = Panel(child=avg_velocity_graph, title='Average Velocity')
# panel3 = Panel(child=avg_group_size_graph, title='Average Group Size')

# Assign the panels to Tabs
# tabs = Tabs(tabs=[panel1, panel2, panel3])
tabs = Tabs(tabs=[panel1])

In [69]:
show(tabs)

ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: avg_num_people [renderer: GlyphRenderer(id='c542ab3e-4ebd-4d47-9020-1b81e71981c5', ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: avg_num_people [renderer: GlyphRenderer(id='494c6ad9-da81-4e0e-8bba-9692021af737', ...)]
